<a href="https://colab.research.google.com/github/huavinhkhang0405/TriTueNhanTao/blob/main/BaoCao_THTTNT_Buoi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MINIMAX

*1. Định nghĩa và Mục đích*

- Minimax là giải thuật là một thuật toán đệ quy lựa chọn bước đi kế tiếp trong một trò chơi có hai người bằng cách định giá trị cho các node trên cây trò chơi sau đó tìm Node có giá trị phù hợp để đi bước tiếp theo.
- Minimax là một phương pháp trong lý thuyết quyết định có mục đích là tối thiểu hóa (minimize) tổn thất vốn được dự tính có thể là tối đa (maximize)

*2. Cơ chế hoạt động*

Quá trình thực thi của Minimax có thể được mô hình hóa qua 3 giai đoạn tuần tự:
1.  **Duyệt cây và Mở rộng không gian trạng thái**: Thuật toán thực hiện tìm kiếm theo chiều sâu (Depth-First Search - DFS) trên cây trò chơi. Từ trạng thái hiện tại (nút gốc), thuật toán sinh ra các trạng thái kế tiếp cho đến khi chạm tới độ sâu giới hạn hoặc đạt tới các trạng thái kết thúc (nút lá/terminal nodes).
2.  **Định giá**: Tại các nút lá, thuật toán áp dụng một hàm định giá



### ALPHA - BETA